In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm

In [ ]:
import tensorflow as tf
print("TENSORFLOW VERSION: ",tf.__version__)

TENSORFLOW VERSION:  2.13.0


<br/>
<br/>
<br/>

# LOADING DATA

In [ ]:
import json
def get_data(path):
  f = open(path)
  data = json.load(f)
  return data['amplitude'][:44100],data['label']

def data_loader(path):
  amplitudes = []
  labels = []
  for file in tqdm(os.listdir(path)):
    amplitude,label = get_data(f'{path}/{file}')
    amplitudes.append(amplitude)
    labels.append(label)

  return np.array(amplitudes),np.array(labels)

In [ ]:
TRAIN_PATH ="./drive/MyDrive/UrbanSoundDataset/train"
VAL_PATH ="./drive/MyDrive/UrbanSoundDataset/val"
TEST_PATH ="./drive/MyDrive/UrbanSoundDataset/test"

train_amp , train_labels = data_loader(TRAIN_PATH)
val_amp , val_labels = data_loader(VAL_PATH)
test_amp , test_labels = data_loader(TEST_PATH)

 83%|████████▎ | 5785/6985 [06:58<03:23,  5.88it/s]

In [ ]:
train_amp.shape

In [ ]:
val_amp.shape

In [ ]:
test_amp.shape

<br/>
<br/>

### LABEL ENCODING

In [ ]:
unique_labels = pd.Series(train_labels).value_counts().index
NUM_OUTPUT_CLASSES = len(unique_labels)
unique_labels

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_labels)

encoded_train_labels = le.transform(train_labels)
encoded_val_labels = le.transform(val_labels)
encoded_test_labels = le.transform(test_labels)

In [ ]:
encoded_train_labels.shape,encoded_val_labels.shape,encoded_test_labels.shape

<br/>
<br/>
<br/>

### CREATING ANN

In [ ]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense , BatchNormalization , Dropout , Flatten
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from IPython.display import clear_output

In [ ]:
# If path does not exist then create the folder
if not os.path.isdir('./drive/MyDrive/UrbanSoundDataset/model_logs'):
    os.mkdir('./drive/MyDrive/UrbanSoundDataset/model_logs')

In [ ]:
def plot_loss_plots(train_loss,val_loss):
  plt.figure(figsize=(8,5),facecolor='#BCD9FC')
  ax = plt.axes()
  ax.set_facecolor("#BCD9FC")
  ax.grid(color="white")

  plt.plot(train_loss,label="training loss",color="green",marker='x')
  plt.plot(val_loss,label="validation loss",color="red",marker='o')
  plt.title("Training Loss VS Validation Loss")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")

  plt.legend();
  plt.show();

In [ ]:
def plot_accuracy_plots(train_accuracy,val_accuracy):
  plt.figure(figsize=(8,5),facecolor='#BCD9FC')
  ax = plt.axes()
  ax.set_facecolor("#BCD9FC")
  ax.grid(color="white")

  plt.plot(train_accuracy,label="training accuracy",color="green",marker='x')
  plt.plot(val_accuracy,label="validation accuracy",color="red",marker='o')
  plt.title("Training Acc VS Validation Acc")
  plt.xlabel("Epochs")
  plt.ylabel("Accuracy")

  plt.legend();
  plt.show();

In [ ]:
def plot_confusion_matrix(y_true,y_pred,title,target_names):
  cm = confusion_matrix(y_true, y_pred)

  plt.figure(figsize=(7,7))
  sns.heatmap(data = cm,annot=True,cmap="coolwarm",xticklabels=target_names, yticklabels=target_names)
  plt.title(title)
  plt.show();

In [ ]:
def save_model_summary(model,filename):
    # If path does not exist then create the folder
    if not os.path.isdir('./drive/MyDrive/UrbanSoundDataset/models'):
        os.mkdir('./drive/MyDrive/UrbanSoundDataset/models')

    with open(f'./drive/MyDrive/UrbanSoundDataset/models/{filename}.txt','w') as f:
        model.summary(print_fn=lambda x: f.write(x + '\n'))
    print("MODEL SUMMARY SAVED")

def save_classification_report(report,filename):
    # If path does not exist then create the folder
    if not os.path.isdir('./drive/MyDrive/UrbanSoundDataset/reports'):
        os.mkdir('./drive/MyDrive/UrbanSoundDataset/reports')

    with open(f'./drive/MyDrive/UrbanSoundDataset/reports/{filename}.txt','w') as f:
        print(report, file=f)
    print("REPORT SAVED")

In [ ]:
def train_and_test(model,filename):

  model.compile( optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy() , metrics = ['accuracy'] )
  model_path = f"./drive/MyDrive/UrbanSoundDataset/{filename}.h5"

  es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3,verbose=1)
  cl = tf.keras.callbacks.CSVLogger(f'./drive/MyDrive/UrbanSoundDataset/model_logs/{filename}.csv')
  mc = tf.keras.callbacks.ModelCheckpoint( filepath=model_path , save_weights_only=True , monitor='val_accuracy', mode='max', save_best_only=True , verbose=1 )


  history = model.fit( train_dataset , validation_data=val_dataset , epochs=1000 , callbacks=[es,cl,mc] , verbose=1)

  # CLEARING THE OUTPUT
  clear_output()

  # LOADING MODEL
  best_model = load_model(model_path)

  # LOSS AND ACCURACY PLOTS
  target_names = le.inverse_transform([i for i in range(NUM_OUTPUT_CLASSES)])
  plot_loss_plots(history.history["loss"],history.history["val_loss"])
  plot_accuracy_plots(history.history["accuracy"],history.history["val_accuracy"])

  test_loss, test_accuracy = best_model.evaluate(test_amp, encoded_test_labels)
  test_pred_labels = best_model.predict(test_amp).argmax(axis=1)
  val_pred_labels = best_model.predict(val_amp).argmax(axis=1)
  print()
  print("TESTING LOSS: ",test_loss)
  print("TESTING ACCURACY: ",test_accuracy)
  print()

  # CONFUSION MATRICES
  plot_confusion_matrix(encoded_test_labels , test_pred_labels , "Test Data Confusion Matrix",target_names)
  plot_confusion_matrix(encoded_val_labels , val_pred_labels , "Validation Data Confusion Matrix",target_names)

  # CLASSIFICATION REPORT
  report = classification_report( encoded_test_labels , test_pred_labels , target_names=target_names )
  print(report)

  # SAVING MODEL SUMMARY AND CLASSIFICATION REPORT
  save_model_summary(model,filename)
  save_classification_report(report,filename)


In [ ]:
model = Sequential([
    Flatten(input_shape=(train_mfccs.shape[1],train_mfccs.shape[2])),

    Dense(512,activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(256,activation='relu'),
    BatchNormalization(),

    Dense(256,activation='relu'),
    BatchNormalization(),

    Dense(64,activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(NUM_OUTPUT_CLASSES,activation='softmax')
])
model.summary()

In [ ]:
filename=f"{model.name}"
train_and_test(model,filename)